In [ ]:
# import cv2 as cv
# import numpy as np
# import matplotlib.pyplot as plt
# import mss.tools
# import time
# import uuid
# import keyboard

# def drive ():
#     time.sleep(10)
#     with mss.mss() as sct:
#         monitor_info = sct.monitors[0]
#         top_margin = (monitor_info["height"] // 3) + 50
#         monitor = {"top": monitor_info["top"] + top_margin, "left": monitor_info["left"] + 55, "width": monitor_info["width"] - 120, "height": (monitor_info["height"] * 2 // 3)-105}

#         # hold space for a bit when you're stopping so it stops
#         while not keyboard.is_pressed("space"):
#             output = f"testingDataset/{str(uuid.uuid4())}.png".format(**monitor)
#             sct_img = sct.grab(monitor)
#             mss.tools.to_png(sct_img.rgb, sct_img.size, output=output)
#             print(f"Captured {output}")
#             time.sleep(1)

    

In [28]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import mss.tools
import time
import uuid
import keyboard
import math

def drive ():
    img_orig = cv.imread('dataset/16cb0d3e-09d2-49db-9a6b-fdc001edcf77.png')
    img_rgb = cv.cvtColor(img_orig, cv.COLOR_BGR2RGB)
    img_blur = cv.blur(img_orig, ksize = (5, 5))
    img_canny = cv.Canny(img_blur, threshold1=121.6, threshold2=180.9)
    cdst = cv.cvtColor(img_rgb, cv.COLOR_RGB2GRAY)
    linesP = cv.HoughLinesP(img_canny, 1, np.pi / 250, 90, None, 94.6, 20.5)
    if linesP is not None:
        for i in range(0, len(linesP)):
            l = linesP[i][0]
            cv.line(cdst, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv.LINE_AA)
    
    y = []
    x = []
    for line in linesP:
        x1, y1, x2, y2 = line[0]
        cv.line(cdst, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw lines in green
        #cv.circle(cdst, (x1, y1), 5, (0, 255, 255), -1)    # Mark starting point in yellow
        #cv.circle(cdst, (x2, y2), 5, (0, 0, 255), -1)      # Mark ending point in red
        y.append(y1)
        y.append(y2)
        x.append(x1)
        x.append(x2)
    
    # Find the indices of the top 2 minimum y values
    top_2_min_indices = sorted(range(len(y)), key=lambda k: y[k])[:2]

    # Get the corresponding x values for the top 2 minimum y values
    matching_x_values = [x[index] for index in top_2_min_indices]
    print(top_2_min_indices)
    print(matching_x_values)

    # Extract coordinates
    x1, y1 = x[0], y[0]
    x2, y2 = x[1], y[1]
    cv.circle(cdst, (x1, y1), 10, (0, 0, 255), -1)      # Mark ending point in red

    cv.circle(cdst, (x2, y2), 10, (0, 0, 255), -1)      # Mark ending point in red

    # Calculate Euclidean distance
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

    print("Distance between the points:", distance)

    center = distance / 2

    cv.imshow("Detected Lines (in red) - Probabilistic Line Transform", cdst)
    cv.waitKey()
drive()

[9, 10]
[665, 1168]
Distance between the points: 522.4327707944823
